In [1]:
# Задание 3
# Ниже приведён код, который демонстрирует реализацию X-learner на наших данных такси. 
# В нём пошагово реализованы все этапы X-learner: от разделения данных на тестовую и контрольную группы до получения финальных предсказаний uplift-эффекта. 
# Запустите его, чтобы увидеть, как алгоритм работает на практике. Для этого вам потребуются данные А/Б теста.

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklift.metrics import uplift_auc_score, qini_auc_score
from catboost import CatBoostClassifier

data = pd.read_csv("ab_results.csv")

# разделим данные на признаки и целевую переменную
X = data.drop(['target'], axis=1)  # все столбцы, кроме целевой переменной
y = data['target']  # целевая переменная (например, количество поездок)

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                    stratify=data[['target', 'treatment']],
                                    random_state=42)

# 1. Разделяем обучающую выборку на treatment=1 и treatment=0
X_treat = X_train[X_train['treatment'] == 1].drop('treatment', axis=1)
y_treat = y_train[X_train['treatment'] == 1]

X_control = X_train[X_train['treatment'] == 0].drop('treatment', axis=1)
y_control = y_train[X_train['treatment'] == 0]

# 2. Обучаем outcome-модели для treatment и control
outcome_treat = CatBoostClassifier(random_state=13, iterations=100, verbose=0)
outcome_control = CatBoostClassifier(random_state=13, iterations=100, verbose=0)

outcome_treat.fit(X_treat, y_treat)
outcome_control.fit(X_control, y_control)

# 3. Строим «псевдотаргеты» (imputed treatment effects)
# Для объектов из treatment-группы: tau0 = y1 - M0(x)
mu0_treat = outcome_control.predict_proba(X_treat)[:, 1]
tau0 = y_treat.values - mu0_treat

# Для объектов из контрольной группы: tau1 = M1(x) - y0
mu1_control = outcome_treat.predict_proba(X_control)[:, 1]
tau1 = mu1_control - y_control.values

# 4. Обучаем effect-модели на псевдотаргетах
effect_0 = LinearRegression()
effect_1 = LinearRegression()
effect_0.fit(X_control, tau1)
effect_1.fit(X_treat, tau0)

# 5. Оцениваем склонность к получению воздействия
propensity = CatBoostClassifier(random_state=13, iterations=100, verbose=0)
propensity.fit(X_train.drop('treatment', axis=1), X_train['treatment'])
e_test = propensity.predict_proba(X_test.drop('treatment', axis=1))[:, 1]

# 6. Получаем предсказания uplift для тестовой выборки
X_test_features = X_test.drop('treatment', axis=1)
tau0_pred = effect_0.predict(X_test_features)
tau1_pred = effect_1.predict(X_test_features)

uplift_pred = e_test * tau1_pred + (1 - e_test) * tau0_pred

# 7. Оценка метрик uplift
uplift_score = uplift_auc_score(
    y_test.values,
    uplift_pred,
    X_test['treatment'].values
)
qini_score = qini_auc_score(
    y_test.values,
    uplift_pred,
    X_test['treatment'].values
)

print(f"Uplift AUC: {uplift_score:.2f}")
print(f"Qini AUC: {qini_score:.2f}")

Uplift AUC: 0.16
Qini AUC: 0.20


In [2]:
# Задание 5
# Запустите код ниже. Данные будут использоваться те же - А/Б теста.

from causalml.inference.meta import BaseXClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklift.metrics import uplift_auc_score, qini_auc_score
from xgboost import XGBClassifier

# используем Random Forest для outcome- и effect-моделей
rf_clf = RandomForestClassifier(
    random_state=42,
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=5,
    n_jobs=-1,
    verbose=0
)
rf_reg = RandomForestRegressor(
    random_state=42,
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=5,
    n_jobs=-1,
    verbose=0
)

# для propensity score используем XGBoost
propensity_model = XGBClassifier(
    random_state=42,
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=-1,
    verbosity=0
)
propensity_model.fit(X_train.drop('treatment', axis=1).values, X_train['treatment'].values)
# получаем propensity score для train и test
p_train = propensity_model.predict_proba(X_train.drop('treatment', axis=1).values)[:, 1]
p_test = propensity_model.predict_proba(X_test.drop('treatment', axis=1).values)[:, 1]

x_learner = BaseXClassifier(
    outcome_learner=rf_clf,
    control_name=0,
    effect_learner=rf_reg,
)

# обучаем X-learner с передачей propensity score
x_learner.fit(
    X=X_train.drop('treatment', axis=1).values,
    treatment=X_train['treatment'].values,
    y=y_train.values,
    p=p_train
)

# получаем uplift-предсказания для тестовой выборки, также передаём propensity score
uplift_pred = x_learner.predict(
    X_test.drop('treatment', axis=1).values,
    p=p_test
)

# оценка метрик uplift
uplift_score = uplift_auc_score(
    y_test.values,
    uplift_pred.squeeze(),
    X_test['treatment'].values
)
qini_score = qini_auc_score(
    y_test.values,
    uplift_pred.squeeze(),
    X_test['treatment'].values
)

print(f"Uplift AUC: {uplift_score:.2f}")
print(f"Qini AUC: {qini_score:.2f}")

Uplift AUC: 0.16
Qini AUC: 0.20


In [3]:
# Задание 7
# Обучите propensity_model (модель вероятности назначения воздействия) с помощью XGBClassifier из библиотеки xgboost. 
# Также сделайте прогноз propensity score для обучающей и тестовой выборок с помощью метода predict_proba.
# Мы предварительно разделили датасет на обучающую и тестовую выборки (X_train, X_test). 
# Для таргета используйте столбец воздействия (treatment_col). Данные можно скачать здесь.

from xgboost import XGBClassifier

# загрузка данных
data = pd.read_csv("yandex_plus.csv")

# разделим данные на признаки и целевую переменную
X = data.drop(['conversion'], axis=1)  
y = data['treatment']  

# применяем маппинг для удобства обучения модели
treatment_mapping = {
    'control': 0,  # 0 соответствует контрольной группе
    'treatment1': 1  # 1 соответствует группе воздействия
}

y = y.map(treatment_mapping)

# разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X.drop(['treatment'], axis=1), y, test_size=0.2, 
                                    stratify=data[['conversion', 'treatment']],
                                    random_state=42)

propensity_model = XGBClassifier(
    random_state=42,
    n_estimators=100,
    max_depth=4,
)

# обучение propensity-модели на обучающей выборке
propensity_model.fit(X_train.values, y_train)

# получаем propensity score для train и test
p_train = propensity_model.predict_proba(X_train.values)[:, 1]
p_test = propensity_model.predict_proba(X_test.values)[:, 1]

print('Среднее значение propensity score для обучающей выборки', round(p_train.mean(), 3))
print('Среднее значение propensity score для тестовой выборки', round(p_test.mean(), 3))

Среднее значение propensity score для обучающей выборки 0.5
Среднее значение propensity score для тестовой выборки 0.498


In [5]:
# Задание 9
# Обучите X-learner c помощью библиотеки causalml. 
# В качестве outcome_learner используйте XGBClassifier из xgboost. В качестве effect_learner используйте XGBRegressor. 
# В качестве propensity scores для обучения модели возьмите прогнозы из предыдущего задания. Не забудьте про random_state=42 для воспроизводимости экспериментов.

from causalml.inference.meta import BaseXClassifier
from xgboost import XGBClassifier, XGBRegressor

# разделим данные заново

X = data.drop(['conversion'], axis=1)  
y = data['conversion']  

# применяем маппинг для удобства обучения модели
treatment_mapping = {
    'control': 0,  # 0 соответствует контрольной группе
    'treatment1': 1  # 1 соответствует группе воздействия
}

X['treatment'] = X['treatment'].map(treatment_mapping)

# разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                    stratify=data[['conversion', 'treatment']],
                                    random_state=42)

# используем Random Forest для outcome- и effect-моделей
xgb_clf = XGBClassifier(
    random_state=42,
    n_estimators=100,
    max_depth=6,
    n_jobs=-1,
    verbose=0
)
xgb_reg = XGBRegressor(
    random_state=42,
    n_estimators=100,
    max_depth=6,
    n_jobs=-1,
    verbose=0
)


x_learner = BaseXClassifier(
    outcome_learner=xgb_clf,
    control_name=0,
    effect_learner=xgb_reg,
)

# обучаем X-learner с передачей propensity score
x_learner.fit(
    X=X_train.values,
    treatment=X_train['treatment'].values,
    y=y_train.values,
    p=p_train,
)


/home/mle-user/mle_projects/mle-sprint5-uplift/.uplift_env/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [05:22:16] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/mle-user/mle_projects/mle-sprint5-uplift/.uplift_env/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [05:22:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [6]:
# Задание 10
# Сделайте прогноз uplift для тестовой выборки и оцените метрики uplift auc и qini auc.

from sklift.metrics import uplift_auc_score, qini_auc_score

# получаем uplift-предсказания для тестовой выборки, также передаём propensity score
uplift_pred = x_learner.predict(
    X_test.values,
    treatment=X_test['treatment'].values,
    p=p_test
)

# оценка метрик uplift
uplift_score = uplift_auc_score(
    y_test.values,
    uplift_pred.squeeze(), 
    X_test['treatment'].values
)
qini_score = qini_auc_score(
    y_test.values,
    uplift_pred.squeeze(), 
    X_test['treatment'].values
)

print(f"Uplift AUC: {uplift_score:.2f}")
print(f"Qini AUC: {qini_score:.2f}")


Uplift AUC: 0.01
Qini AUC: 0.02
